In [1]:
import sys 
sys.path.append('/home/samuel.varga/python_packages/wofs_ml_severe')
sys.path.append('/home/samuel.varga/python_packages/WoF_post')
#sys.path.append('/home/samuel.varga/python_packages/wofs_ml_severe')
sys.path.append('/home/samuel.varga/projects/2to6_hr_severe_wx/mlw2w')
sys.path.append('/home/samuel.varga/python_packages/MontePython/')
import numpy as np
import xarray as xr
from glob import glob
from ml_2to6_data_pipeline import (GridPointExtracter,
                                                       subsampler, 
                                                       load_dataset)
from os.path import join
# Import packages 
import pandas as pd
import numpy as np

# Plotting code imports 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

# We add the github package to our system path so we can import python scripts for that repo. 
import sys
sys.path.append('/home/samuel.varga/projects/2to6_hr_severe_wx/')
from main.io import load_ml_data, load_bl_data
from bayeshist import bayesian_histogram, plot_bayesian_histogram
from wofs_ml_severe.data_pipeline.storm_report_loader import StormReportLoader
from wofs.plotting.util import decompose_file_path
from wofs.verification.lsrs.get_storm_reports import StormReports

lookup_file: /home/samuel.varga/python_packages/WoF_post/wofs/data/psadilookup.dat


In [2]:
FRAMEWORK='POTVIN'
for TIMESCALE in ['2to6']:
    base_path = f'/work/samuel.varga/data/{TIMESCALE}_hr_severe_wx/sfe_prep'
    X,y,metadata = load_ml_data(base_path=base_path, 
                                mode=None, 
                                target_col='hail_severe__36km',
                               FRAMEWORK=FRAMEWORK, full_9km=True,
                               TIMESCALE=TIMESCALE)
    dates=np.sort(pd.unique(metadata['Run Date']))
    print(dates)
    print(len(dates))

['20190501' '20190502' '20190503' '20190506' '20190507' '20190508'
 '20190509' '20190510' '20190513' '20190514' '20190515' '20190516'
 '20190517' '20190518' '20190520' '20190521' '20190522' '20190523'
 '20190524' '20190525' '20190526' '20190528' '20190529' '20190530'
 '20200501' '20200504' '20200505' '20200506' '20200507' '20200508'
 '20200513' '20200515' '20200518' '20200519' '20200520' '20200521'
 '20200522' '20200526' '20200527' '20200528' '20200529' '20210503'
 '20210504' '20210505' '20210506' '20210507' '20210510' '20210512'
 '20210513' '20210514' '20210517' '20210518' '20210519' '20210520'
 '20210521' '20210523' '20210524' '20210525' '20210526' '20210527'
 '20210528' '20220502' '20220503' '20220504' '20220505' '20220506'
 '20220509' '20220510' '20220511' '20220512' '20220513' '20220516'
 '20220517' '20220518' '20220519' '20220520' '20220523' '20220524'
 '20220525' '20220526' '20220527' '20220528' '20220529' '20220530'
 '20220531' '20230501_d1' '20230502_d1' '20230503_d1' '2023050

In [16]:
len(metadata.groupby(['Run Date','Init Time']).sum())
#np.unique(metadata['Init Time'])

644

In [3]:
#metadata[metadata['Run Date']=='20180502']['Init Time'].unique()
#temp=metadata[["2018" in date for date in metadata['Run Date']]].groupby('Run Date').nunique().reset_index()
#temp=metadata[["2021" in date for date in metadata['Run Date']]].reset_index()['Init Time'].unique() #.nunique().reset_index()
#print(len(temp['Init Time']))
#print(temp['Init Time'].sum())
#temp

In [7]:
FRAMEWORK='POTVIN'; TIMESCALE='0to3'
base_path = f'/work/samuel.varga/data/{TIMESCALE}_hr_severe_wx/{FRAMEWORK}'
X,y,metadata = load_ml_data(base_path=base_path, 
                            mode=Non, 
                            target_col='hail_severe__36km',
                           FRAMEWORK=FRAMEWORK,
                           TIMESCALE=TIMESCALE, full_9km=False) #What scales are available for targets: 9, 18, 36
dates=np.sort(pd.unique(metadata['Run Date']))
init_time_='0000'
dates

array(['20180511', '20180514', '20180521', '20180531', '20190501',
       '20190502', '20190508', '20190510', '20190515', '20190520',
       '20190521', '20190523', '20190526', '20200506', '20200507',
       '20200508', '20200519', '20200529', '20210504', '20210507',
       '20210510', '20210513', '20210520', '20210521', '20210523'],
      dtype=object)

In [ ]:
  '20180524',
       '20180530', 
       '20190513', '20190516', '20190517', '20190520', '20190526',
       '20200506', '20200518', '20200522', '20200528', '20210503',
       '20210504', '20210510', '20210513', '20210518', '20210528'

In [9]:
import datetime as dt
print('{:^8}|{}|{}|{}|'.format('Date','Wind','Hail','Torn'))
print('--------|----|----|----|')
for date in dates:
    indir = glob(f'/work/mflora/SummaryFiles/{date}/{init_time_}/wofs_ENS_24*')[0]
    # Get the storm reports. 
    comps = decompose_file_path(indir)
    #init_time = comps['VALID_DATE']+comps['VALID_TIME']

    #init_time = comps['VALID_DATE']+comps['INIT_TIME']
    start_time=(pd.to_datetime(comps['VALID_DATE']+comps['INIT_TIME'])+dt.timedelta(minutes=int(comps['TIME_INDEX'])*5)).strftime('%Y%m%d%H%M')

    forecast_length = 180 if TIMESCALE=='0to3' else 240



    report = StormReportLoader(
                reports_path = '/work/mflora/LSRS/StormEvents_2017-2022.csv',
                report_type='NOAA',
                initial_time=start_time, 
                forecast_length=forecast_length, 
                err_window=15,               
            )

    ds = xr.load_dataset(indir, decode_times=False)
    points = report()
    print('{}| {:02} | {:02} | {:02} |'.format(date, points['wind'].shape[0], points['hail'].shape[0], points['tornado'].shape[0] ))

  Date  |Wind|Hail|Torn|
--------|----|----|----|
20180511| 00 | 00 | 00 |
20180514| 36 | 68 | 14 |
20180521| 06 | 06 | 06 |
20180531| 20 | 22 | 00 |
20190501| 04 | 08 | 00 |
20190502| 04 | 04 | 02 |
20190508| 08 | 18 | 00 |
20190510| 02 | 00 | 00 |
20190515| 14 | 14 | 00 |
20190520| 58 | 54 | 18 |
20190521| 08 | 00 | 00 |
20190523| 38 | 28 | 08 |
20190526| 104 | 38 | 28 |
20200506| 00 | 00 | 00 |
20200507| 04 | 62 | 00 |
20200508| 04 | 04 | 00 |
20200519| 04 | 06 | 02 |
20200529| 02 | 00 | 00 |
20210504| 47 | 00 | 00 |
20210507| 00 | 00 | 00 |
20210510| 02 | 09 | 00 |
20210513| 00 | 01 | 00 |
20210520| 03 | 03 | 02 |
20210521| 02 | 00 | 00 |
20210523| 25 | 03 | 00 |


# 2-6 Hr Test Dates

In [4]:
FRAMEWORK='POTVIN'; TIMESCALE='2to6'
base_path = f'/work/samuel.varga/data/{TIMESCALE}_hr_severe_wx/{FRAMEWORK}'
X,y,metadata = load_ml_data(base_path=base_path, 
                            mode='test', 
                            target_col='hail_severe__36km',
                           FRAMEWORK=FRAMEWORK,
                           TIMESCALE=TIMESCALE, full_9km=True)
dates=np.sort(pd.unique(metadata['Run Date'])) #Grab the dates in the testing set
init_time_='0000' #20200519, 20180503, 20180511,  20180523, 20190520, 20210505, 20210520, 
dates

array(['20180511', '20180514', '20180521', '20180531', '20190501',
       '20190502', '20190508', '20190510', '20190515', '20190520',
       '20190521', '20190523', '20190526', '20200506', '20200507',
       '20200508', '20200519', '20200529', '20210504', '20210507',
       '20210510', '20210513', '20210520', '20210521', '20210523'],
      dtype=object)

In [11]:
FRAMEWORK='ADAM'; TIMESCALE='2to6'
base_path = f'/work/samuel.varga/data/{TIMESCALE}_hr_severe_wx/{FRAMEWORK}'
X,y,metadata = load_ml_data(base_path=base_path, 
                            mode='test', 
                            target_col='hail_severe__36km',
                           FRAMEWORK=FRAMEWORK,
                           TIMESCALE=TIMESCALE, Big=False)
dates=np.sort(pd.unique(metadata['Run Date'])) #Grab the dates in the testing set
init_time_='0000' #20200519, 20180503, 20180511,  20180523, 20190520, 20210505, 20210520, 
dates

Appending init time to predictors


array(['20180511', '20180514', '20180521', '20180531', '20190501',
       '20190502', '20190508', '20190510', '20190515', '20190520',
       '20190521', '20190523', '20190526', '20200506', '20200507',
       '20200508', '20200519', '20200529', '20210504', '20210507',
       '20210510', '20210513', '20210520', '20210521', '20210523'],
      dtype=object)

In [5]:
'20180501', '20180510', '20180515', '20180524', '20180528',
       '20180530', '20190501', '20190506', '20190509', '20190510',
       '20190513', '20190516', '20190517', '20190520', '20190526',
       '20200506', '20200518', '20200522', '20200528', '20210503',
       '20210504', '20210510', '20210513', '20210518', '20210528']

IndentationError: unexpected indent (424570557.py, line 2)

In [6]:
import datetime as dt
print('{:^8}|{}|{}|{}|'.format('Date','Wind','Hail','Torn'))
print('--------|----|----|----|')
for date in dates:
    indir = glob(f'/work/mflora/SummaryFiles/{date}/{init_time_}/wofs_ENS_24*')[0]
    # Get the storm reports. 
    comps = decompose_file_path(indir)
   # init_time = comps['VALID_DATE']+comps['VALID_TIME']

    #init_time = comps['VALID_DATE']+comps['INIT_TIME']

    start_time=(pd.to_datetime(comps['VALID_DATE']+comps['INIT_TIME'])+dt.timedelta(minutes=int(comps['TIME_INDEX'])*5)).strftime('%Y%m%d%H%M')

    forecast_length = 180 if TIMESCALE=='0to3' else 240



    report = StormReportLoader(
                reports_path = '/work/mflora/LSRS/StormEvents_2017-2022.csv',
                report_type='NOAA',
                initial_time=start_time, 
                forecast_length=forecast_length, 
                err_window=15,               
            )

    ds = xr.load_dataset(indir, decode_times=False)
    points = report()
    print('{}| {:02} | {:02} | {:02} |'.format(date, points['wind'].shape[0], points['hail'].shape[0], points['tornado'].shape[0] ))

  Date  |Wind|Hail|Torn|
--------|----|----|----|
20180511| 00 | 00 | 00 |
20180514| 36 | 72 | 14 |
20180521| 06 | 06 | 06 |
20180531| 24 | 22 | 00 |
20190501| 04 | 12 | 00 |
20190502| 04 | 04 | 02 |
20190508| 10 | 22 | 00 |
20190510| 02 | 00 | 00 |
20190515| 14 | 14 | 00 |
20190520| 60 | 54 | 18 |
20190521| 10 | 00 | 02 |
20190523| 38 | 28 | 08 |
20190526| 136 | 42 | 34 |
20200506| 00 | 00 | 00 |
20200507| 06 | 68 | 00 |
20200508| 04 | 04 | 00 |
20200519| 04 | 06 | 02 |
20200529| 02 | 00 | 00 |


IndexError: list index out of range

In [14]:
X.columns

Index(['uh_2to5_instant__time_max__4km__ens_16th',
       'uh_0to2_instant__time_max__4km__ens_16th',
       'wz_0to2_instant__time_max__4km__ens_16th',
       'comp_dz__time_max__4km__ens_16th', 'ws_80__time_max__4km__ens_16th',
       'hailcast__time_max__4km__ens_16th', 'w_up__time_max__4km__ens_16th',
       'okubo_weiss__time_max__4km__ens_16th',
       'uh_2to5_instant__time_max__4km__ens_max',
       'uh_0to2_instant__time_max__4km__ens_max',
       'wz_0to2_instant__time_max__4km__ens_max',
       'comp_dz__time_max__4km__ens_max', 'ws_80__time_max__4km__ens_max',
       'hailcast__time_max__4km__ens_max', 'w_up__time_max__4km__ens_max',
       'okubo_weiss__time_max__4km__ens_max',
       'uh_2to5_instant__time_max__4km__ens_mean__smoothed',
       'uh_0to2_instant__time_max__4km__ens_mean__smoothed',
       'wz_0to2_instant__time_max__4km__ens_mean__smoothed',
       'comp_dz__time_max__4km__ens_mean__smoothed',
       'ws_80__time_max__4km__ens_mean__smoothed',
       'hailc